In [3]:
import os
import pickle

import pandas as pd

from tqdm import tqdm

from wbapi import WbApi

In [4]:
with open('token.txt') as f:
    token = f.read()

In [5]:
with WbApi.create_wb_api(token) as wb_api:
    print(wb_api.get_cards_by_vendors('SR1-|PU|-C826///1'))

[{'imtID': 113425616, 'nmID': 131701653, 'vendorCode': 'SR1-|PU|-C826///1', 'mediaFiles': ['', 'https://basket-10.wb.ru/vol1317/part131701/131701653/images/big/1.jpg'], 'sizes': [{'techSize': '0', 'skus': ['SR1-|PU|-C826///1'], 'chrtID': 227065322, 'wbSize': '', 'price': 6837}], 'characteristics': [{'Ширина упаковки': 15}, {'Высота упаковки': 10}, {'Длина упаковки': 15}, {'Комплектация': ['1']}, {'ОЕМ номер': ['164003VD0C', 'MFE 1653 MB', 'J1331052', '3ECO097', 'FE0069', '37NI029', '1640000Q3A', '1500166', 'PAL2-6009', 'CFF100659', '101326', 'E216L', 'E437KP D266', 'F320701', 'LF-779', '28136-LF-PCS-MS', '2136719', 'GS014389E', 'C11574', '28136/1-LF-PCS-MS', 'F140', 'SKFF-0870115', '1640000Q2E', '16400']}, {'Артикул производителя': ['SR1-|PU|-C826///1']}, {'Бренд': 'PURFLUX'}, {'Предмет': 'Ремкомплекты автомобильные'}, {'Наименование': 'Топливный фильтр, 164003VD0C !'}, {'Описание': 'Топливный фильтр.  9. Внешний диаметр 81мм. Внутренний диаметр 11мм. высота 118мм. Фильтр топливный DAC

In [9]:
df = pd.read_excel('data\\vendors.xlsx')

In [15]:
df.values.T[0].tolist()

['AA9-|BO|-3397118996///1',
 'MB9-|BO|-3397001984///1',
 'FR1-|BO|-3397008997///1',
 'AA9-|BO|-3397118938///1',
 'FA9-|BO|-3397118913///1',
 'FA9-|BO|-3397004628///1',
 'FR1-|BO|-3397007294///1',
 'FR1-|BO|-3397014212///1',
 'FR1-|BO|-3397014211///1',
 'AK9-|BO|-1457433593///1',
 'MB9-|BO|-3397118996///1',
 'FA9-|BO|-3397007565///1',
 'AK9-|BO|-0986594590///1',
 'AE9-|BO|-3397005807///1',
 'AK9-|BO|-3397018964///1',
 'FA9-|BO|-3397005808///1',
 'AE9-|BO|-3397118972///1',
 'MB9-|BO|-3397004802///1',
 'FR1-|BO|-3397007589///1',
 'FR1-|BO|-3397011628///1',
 'AK9-|BO|-3397001543///1']

In [8]:
cards_to_modify = []
for file_name in os.listdir('data/cards_empty/'):
    with open(f'data/cards_empty/{file_name}', 'rb') as f:
        cards_to_modify.extend(list(pickle.load(f)))

In [10]:
len(cards_to_modify)

613507